# Incremental Learning Model
This origin comming from https://www.kaggle.com/code/lblhandsome/optiver-robust-best-single-model/notebook

# Init

## Imports

In [1]:
from pathlib import Path
import os
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import gc  # Garbage collection for memory management
import os  # Operating system-related functions
import time  # Time-related functions
import warnings  # Handling warnings
from itertools import combinations  # For creating combinations of elements
from warnings import simplefilter  # Simplifying warning handling
import joblib  # For saving and loading models
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and analysis
from sklearn.metrics import mean_absolute_error  # Metric for evaluation
from sklearn.model_selection import KFold, TimeSeriesSplit  # Cross-validation techniques
from concurrent.futures import ThreadPoolExecutor
from numba import njit, prange  # Compiling Python code for performance

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Global params

In [2]:
# Disable warnings to keep the code clean
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

warnings.filterwarnings("ignore")

seed = 2023
DATA_COUNT_IN_SAME_BUCKET = 55 # 同じbucket内のデータ数

# For kaggle environment
if os.environ.get("KAGGLE_DATA_PROXY_TOKEN") != None:
    BASE_OUTPUT_PATH = Path(f'/kaggle/working')
    BASE_INPUT_PATH = Path(f'/kaggle/input/optiver-trading-at-the-close')
    TRAIN_FILE = Path(f'{BASE_INPUT_PATH}/train.csv')
    TEST_FILE = Path(f'{BASE_INPUT_PATH}/example_test_files/test.csv')
    
    IS_LOCAL = False # If kaggle environment, set False
    IS_INFER = True # If kaggle environment, set True
    IS_USE_SAVED_MODEL = False # Use saved model or not
    IS_MIN_LEARN = False # Use min learning or not
    USE_OPTUNA = False # Use optuna or not
    USE_CONTINUOUS_UPDATE = True # Use test date on train or not
    USE_ALL_FEATUTES = True # Use all features or not
    USE_REVEALED_TARGETS = True # Use revealed targets or not
    USE_INDEX = True # Use index or not
    IS_DEBUG = True
    USE_ADDITIONAL_TRAIN = True
    NUM_THREADS = 4

    SAMPLE_SUBMISSION_FILE = Path(f'{BASE_INPUT_PATH}/example_test_files/sample_submission.csv')
    REVEALED_TARGETS_FILE = Path(f'{BASE_INPUT_PATH}/example_test_files/revealed_targets.csv')

    stopping_rounds = 100 # early_stopping用コールバック関数
    num_boost_round = 5000 # 計算回数
    update_num_boost_round = 5000 # 再学習の計算回数
    num_folds = 5 # クロスバリデーションの分割数
    continuos_dataset_span = 20 # DATA_COUNT_IN_SAME_BUCKET * continuos_dataset_span が更新の使用する対象のデータ
    continuos_train_span = 3 # DATA_COUNT_IN_SAME_BUCKET * continuos_train_span が更新の頻度

    DEVICE = 'gpu' # cpu or gpu
    OPTUNA_TIME_BUDGET = 60 * 60 * 4 # 1 hours
    TARGET_STOCK_IDS = [0, 1]

    optuna_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'verbosity': -1,
        'deterministic':True, #再現性確保用のパラメータ
        'force_row_wise':True,  #再現性確保用のパラメータ
        'device': DEVICE
    }

    # サンプルのパラメータ
    """
    lgb_params = {
        "objective": "mae",
        "n_estimators": 5500,
        "num_leaves": 128,
        "subsample": 0.6,
        "colsample_bytree": 0.8,
        "learning_rate": 0.00005,
        'max_depth': 11,
        "n_jobs": 4,
        "device": DEVICE,
        "verbosity": -1,
        "importance_type": "gain",
    }
    """
    
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression', 
        'metric': 'rmse', 
        'verbosity': -1, 
        'device': DEVICE,
        'feature_pre_filter': False, 
        'lambda_l1': 0.0,
        'lambda_l2': 0.0,
        'num_leaves': 31, 
        'feature_fraction': 0.8, 
        'bagging_fraction': 1.0, 
        'bagging_freq': 0, 
        'min_child_samples': 20,
        'seed': seed,                       # シード値
    }

    
   # こっちのパラメータの方が、計算時間がかかる
    """
    lgb_params = {
        'task': 'train',                   # 学習
        'objective': 'regression',                # 目的関数の種類。ここでは回帰タスクを指定
        'metric': 'rmse',                          # 評価指標
        'boosting_type': 'gbdt',                  # ブースティングタイプ。勾配ブースティング決定木
        "n_estimators": 32,                        # ブースティングに使用する木の数。多いほど性能が向上するが計算コストが増加
        "num_leaves": 64,                         # 木に存在する最大の葉の数。大きい値は精度を向上させるが過学習のリスクが増加
        "subsample": 0.8,                         # 各木のトレーニングに使用されるデータの割合。過学習を防ぐために一部のデータをサンプリング
        "colsample_bytree": 0.8,                  # 木を構築する際に使用される特徴の割合。特徴のサブセットを使用し過学習を防ぐ
        "learning_rate": 0.01,                 # 学習率。小さい値は堅牢なモデルを生成するが収束に時間がかかる
        'max_depth': 32,                           # 木の最大の深さ。深い木は複雑なモデルを作成するが過学習のリスクがある
        "device": DEVICE,                         # トレーニングに使用するデバイス（CPUまたはGPU）
        "verbosity": -1,                          # LightGBMのログ出力のレベル。-1はログを出力しないことを意味する
       # "importance_type": "gain",                # 特徴重要度を計算する際の指標。"gain"は分割による平均情報利得
        'lambda_l1': 0.5,                         # L1正則化項の係数。過学習を防ぐためにモデルの複雑さにペナルティを課す
        'lambda_l2': 0.5,                         # L2正則化項の係数。同じく過学習を防ぐ
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
    }
    """

# For local environment
else:
    BASE_OUTPUT_PATH = Path(f'../output')
    BASE_INPUT_PATH = Path(f'../kaggle/input/optiver-trading-at-the-close')
    TRAIN_FILE = Path(f'{BASE_INPUT_PATH}/train.csv')
    TEST_FILE = Path(f'{BASE_INPUT_PATH}/test.csv')

    SAMPLE_SUBMISSION_FILE = Path(f'{BASE_INPUT_PATH}/sample_submission.csv')
    REVEALED_TARGETS_FILE = Path(f'{BASE_INPUT_PATH}/revealed_targets.csv')

    IS_LOCAL = True
    IS_INFER = True
    IS_USE_SAVED_MODEL = False # Use saved model or not
    IS_MIN_LEARN = True
    USE_OPTUNA = False # Use optuna or not
    USE_ALL_FEATUTES = True # Use all features or not
    USE_CONTINUOUS_UPDATE = True # Use test date on train or not
    USE_REVEALED_TARGETS = True # Use revealed targets or not 
    USE_INDEX = True # Use index or not
    IS_DEBUG = True
    USE_ADDITIONAL_TRAIN = True
    TARGET_STOCK_IDS = [0]
    NUM_THREADS = 2

    # For training
    stopping_rounds = 1 # early_stopping用コールバック関数
    num_boost_round = 1 # 計算回数
    update_num_boost_round = 1
    num_folds = 2 # クロスバリデーションの分割数
    continuos_dataset_span = 3 # DATA_COUNT_IN_SAME_BUCKET * continuos_dataset_span が更新の使用する対象のデータ
    continuos_train_span = 2 # DATA_COUNT_IN_SAME_BUCKET * continuos_train_span が更新の頻度

    DEVICE = 'cpu' # cpu or gpu
    OPTUNA_TIME_BUDGET = 60 # 1 min

    optuna_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'verbosity': -1,
        'deterministic':True, #再現性確保用のパラメータ
        'force_row_wise':True,  #再現性確保用のパラメータ
        'device': DEVICE
    }

    lgb_params = {
        'task': 'train',                   # 学習
        'boosting_type': 'gbdt',           # GBDT
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'learning_rate': 0.01,             # 学習率
        'lambda_l1': 0.5,                  # L1正則化項の係数
        'lambda_l2': 0.5,                  # L2正則化項の係数
        'num_leaves': 10,                  # 最大葉枚数
        'feature_fraction': 0.5,           # ランダムに抽出される列の割合
        'bagging_fraction': 0.5,           # ランダムに抽出される標本の割合
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
        "device": DEVICE,
        'verbosity': -1
    }


print(f"BASE_OUTPUT_PATH: {BASE_OUTPUT_PATH}")
print(f"BASE_INPUT_PATH: {BASE_INPUT_PATH}")
print(f"TRAIN_FILE: {TRAIN_FILE}")
print(f"TEST_FILE: {TEST_FILE}")
print(f"IS_LOCAL: {IS_LOCAL}")
print(f"IS_INFER: {IS_INFER}")
print(f"IS_USE_SAVED_MODEL: {IS_USE_SAVED_MODEL}")
print(f"IS_MIN_LEARN: {IS_MIN_LEARN}")
print(f"USE_OPTUNA: {USE_OPTUNA}")
print(f"USE_CONTINUOUS_UPDATE: {USE_CONTINUOUS_UPDATE}")
print(f"USE_ALL_FEATUTES: {USE_ALL_FEATUTES}")
print(f"USE_REVEALED_TARGETS: {USE_REVEALED_TARGETS}")
print(f"USE_INDEX: {USE_INDEX}")

BASE_OUTPUT_PATH: /kaggle/working
BASE_INPUT_PATH: /kaggle/input/optiver-trading-at-the-close
TRAIN_FILE: /kaggle/input/optiver-trading-at-the-close/train.csv
TEST_FILE: /kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
IS_LOCAL: False
IS_INFER: True
IS_USE_SAVED_MODEL: False
IS_MIN_LEARN: False
USE_OPTUNA: False
USE_CONTINUOUS_UPDATE: True
USE_ALL_FEATUTES: True
USE_REVEALED_TARGETS: True
USE_INDEX: True


# Functions

## Memory Functions

In [3]:
%%time 

from gc import collect;
from psutil import Process;
from os import system, getpid, walk;

# Defining global configurations and functions:-

    
def GetMemUsage():
    """
    This function defines the memory usage across the kernel. 
    Source-
    https://stackoverflow.com/questions/61366458/how-to-find-memory-usage-of-kaggle-notebook
    """;
    
    pid = getpid();
    py = Process(pid);
    memory_use = py.memory_info()[0] / 2. ** 30;
    return f"RAM memory GB usage = {memory_use :.4}";


collect();
print(GetMemUsage())

RAM memory GB usage = 0.727
CPU times: user 196 ms, sys: 999 µs, total: 197 ms
Wall time: 195 ms


In [4]:
# 🧹 Function to reduce memory usage of a Pandas DataFrame
def reduce_mem_usage(df, name: str, show_optimization: bool = False):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    
    # 📏 Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    if show_optimization:
        print(f"Memory usage of {name} is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    # 🔄 Return the DataFrame with optimized memory usage

    start_mem = df.memory_usage().sum() / 1024**2
    return df

## API Function

In [5]:
%%time 

from typing import Sequence, Tuple
import pandas as pd

# for local execution
class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = [TEST_FILE, REVEALED_TARGETS_FILE, SAMPLE_SUBMISSION_FILE]
        self.group_id_column: str = 'time_id'
        self.export_group_id_column: bool = True
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open('submission.csv', 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'

def make_env():
    return MockApi()

CPU times: user 83 µs, sys: 0 ns, total: 83 µs
Wall time: 87 µs


## Pandas Functions

In [6]:
def pd_display_max():
    pd.set_option('display.max_rows', None)  # 行の最大表示数を無制限に設定
    pd.set_option('display.max_columns', None)  # 列の最大表示数を無制限に設定
    pd.set_option('display.width', None)  # 表示幅を拡張
    pd.set_option('display.max_colwidth', None)  # 列の幅を最大に設定

def pd_clear_display_max():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.width', None)  # 表示幅を拡張
    pd.set_option('display.max_colwidth', None)  # 列の幅を最大に設定

## Sorting Functions

In [7]:
def default_sort(df):
    return df.sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)

# Generationg train dataset

In [8]:
def load_train_dataset():
    df = pd.read_csv(TRAIN_FILE)
    # 🧹 Remove rows with missing values in the "target" column
    df = df.dropna(subset=["target"])
    # 🔁 Reset the index of the DataFrame and apply the changes in place
    df.reset_index(drop=True, inplace=True)
    return df

def load_test_dataset():
    df_test = pd.read_csv(TEST_FILE)
    
    df_revealed_targets = pd.read_csv(REVEALED_TARGETS_FILE)
    df_revealed_targets = df_revealed_targets.dropna(subset=['date_id', 'seconds_in_bucket', 'stock_id'])
    df_revealed_targets['revealed_date_id'] = df_revealed_targets['revealed_date_id'].astype(int).astype(str)
    df_revealed_targets['seconds_in_bucket'] = df_revealed_targets['seconds_in_bucket'].astype(int).astype(str)
    df_revealed_targets['stock_id'] = df_revealed_targets['stock_id'].astype(int).astype(str)  # Converting to int first to remove any decimal points
    df_revealed_targets['date_id'] = df_revealed_targets['date_id'].astype(int).astype(str)
    df_revealed_targets['row_id'] = df_revealed_targets['date_id'] + '_' + df_revealed_targets['seconds_in_bucket'] + '_' + df_revealed_targets['stock_id']
    df_revealed_targets['revealed_row_id'] = df_revealed_targets['revealed_date_id'] + '_' + df_revealed_targets['seconds_in_bucket'] + '_' + df_revealed_targets['stock_id']

    # USE_CONTINUOUS_UPDATEが有効の時、cacheのrow_idとdf_revealed_targetsのrevealed_row_idをleft joinする
    if USE_CONTINUOUS_UPDATE:
        df_r = df_revealed_targets[['revealed_row_id', 'revealed_target']]
        df_r.rename(columns={'revealed_target': 'target'}, inplace=True)
        df_r.rename(columns={'revealed_row_id': 'row_id'}, inplace=True)
        df_test = pd.merge(df_test, df_r, how='left', on='row_id')
        
    # USE_REVEALED_TARGETSが有効の時、cacheのrow_idとdf_revealed_targetsのrow_idをleft joinする
    if USE_REVEALED_TARGETS:
        df_r = df_revealed_targets[['row_id', 'revealed_target']]
        df_test = pd.merge(df_test, df_r, how='left', on='row_id')
        
    df_test = df_test.dropna(subset=["target"])
    df_test.reset_index(drop=True, inplace=True)
    return df_test

In [9]:
%%time
# Check if the code is running in offline or online mode
print("Load train dataset")

df_train = load_train_dataset()

if IS_MIN_LEARN:
    print("MIN LEARN MODE :", TARGET_STOCK_IDS)
    # In local mode, stock id TARGET_STOCK_ID is used for training
    df_train = df_train[df_train["stock_id"].isin(TARGET_STOCK_IDS)]
    
features = [c for c in df_train.columns if c not in ["row_id", "target", "time_id", "row_id", "date_id", "currently_scored"]]
print(features)

collect();
print(GetMemUsage())

Load train dataset
['stock_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap']
RAM memory GB usage = 2.004
CPU times: user 12.3 s, sys: 1.38 s, total: 13.6 s
Wall time: 17.9 s


# Generate Featuers

## Step1. Basic Features

In [10]:
# Function to compute triplet imbalance in parallel using Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # 🔁 Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        # 🔁 Loop through rows of the DataFrame
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            # 🚫 Prevent division by zero
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

# 📈 Function to calculate triplet imbalance for given price data and a DataFrame
def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance using the Numba-optimized function
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

# Function to generate imbalance features
def imbalance_features(df):
    def __imbalance_features(df):
        # Define lists of price and size-related column names
        prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
        sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

        # V1 features
        # Calculate various features using Pandas eval function
        df["volume"] = df.eval("ask_size + bid_size")
        df["mid_price"] = df.eval("ask_price + bid_price")/2
        df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
        df["matched_imbalance"] = df.eval("imbalance_size-matched_size")/df.eval("matched_size+imbalance_size")
        df["size_imbalance"] = df.eval("bid_size / ask_size")
        
        # Create features for pairwise price imbalances
        for c in combinations(prices, 2):
            df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
            
        # V2 features
        # Calculate additional features
        df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
        df["price_spread"] = df["ask_price"] - df["bid_price"]
        df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
        df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
        df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
        df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
        # Calculate the imbalance ratio
        df['match_balance'] = ( df['matched_size']  + (df['imbalance_buy_sell_flag'] * df['imbalance_size'])) / df['matched_size']
        return df

    if DEVICE == 'gpu':
        import cudf
        df = cudf.from_pandas(df)
        df = __imbalance_features(df)
        df = df.to_pandas()
    else:
        df = __imbalance_features(df)
    # Replace infinite values with 0
    return df.replace([np.inf, -np.inf], 0)

def numba_imb_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    # Calculate triplet imbalance features using the Numba-optimized function
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    return df

# 📅 Function to generate time and stock-related features
def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  # Seconds
    df["minute"] = df["seconds_in_bucket"] // 60  # Minutes

    # Map global features to the DataFrame
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

# 🚀 Function to generate all features by combining imbalance and other features
def generate_basic_features(df):
    prev_cols = list(df.columns)

    # Generate imbalance features
    df = imbalance_features(df)
    df = numba_imb_features(df)
    df = other_features(df)

    df = default_sort(df)    
    
    df = reduce_mem_usage(df, "generate_basic_features")
    collect()  # Perform garbage collection to free up memory
    return df

In [11]:
%%time

print("Step1. Generate general Global Stock ID Features and basic features")
prev_cols = list(df_train.columns)
global_stock_id_feats = {
    "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
    "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
    "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
    "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
    "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
    "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
}

df_train = generate_basic_features(df_train)

generated_feature_name = list(set(df_train.columns) - set(prev_cols))
features += generated_feature_name
print(generated_feature_name)

collect()
print(GetMemUsage())

Step1. Generate general Global Stock ID Features and basic features
['all_sizes_skew', 'spread_intensity', 'reference_price_near_price_imb', 'global_ptp_price', 'global_std_price', 'all_prices_mean', 'reference_price_bid_price_imb', 'matched_size_bid_size_imbalance_size_imb2', 'far_price_ask_price_imb', 'price_spread', 'match_balance', 'far_price_near_price_imb', 'global_median_price', 'reference_price_ask_price_imb', 'bid_price_wap_imb', 'global_median_size', 'near_price_bid_price_imb', 'ask_price_wap_imb', 'seconds', 'all_sizes_kurt', 'dow', 'near_price_ask_price_imb', 'bid_size_ask_size_imbalance_size_imb2', 'ask_price_bid_price_wap_imb2', 'minute', 'depth_pressure', 'size_imbalance', 'market_urgency', 'global_std_size', 'all_sizes_mean', 'ask_price_bid_price_reference_price_imb2', 'near_price_wap_imb', 'global_ptp_size', 'all_sizes_std', 'far_price_bid_price_imb', 'bid_price_wap_reference_price_imb2', 'far_price_wap_imb', 'matched_imbalance', 'matched_size_ask_size_imbalance_size_i

## Step2. Enhance features

In [12]:
%%time

print("Step2. Generate enhanced features")
prev_cols = list(df_train.columns)

@njit()
def cal_diff(x, window):
    # pands diffより遅い
    # xの長さと同じ大きさの配列を作成し、初期値をNaNに設定
    log_diff = np.full(x.shape, np.nan)
    # 指定されたwindowに基づいて差分を計算
    for i in range(window, len(x)):
        log_diff[i] = x[i] - x[i - window]

    return log_diff

#@njit()
#@njit(parallel=True)
@njit()
def cal_vix(x, window, offset=0):
    log_x = np.log(x + offset)
    log_diff = np.empty(log_x.shape)
    roll_std = np.empty(log_diff.shape)

    for i in prange(1, len(log_x)):
        log_diff[i] = log_x[i] - log_x[i - 1]
    
    # ローリング標準偏差を計算
    # jitを使わない場合、roll_std[i] = np.std(log_diff[i-window+1:i+1], ddof=1)と書ける(不偏推定量を使うためddof=1)
    # jitを使う場合、ddof=1は使えないので、標準偏差の計算を自分で実装する
    for i in prange(window, len(log_diff)):
        window_values = log_diff[i-window+1:i+1]
        mean = np.mean(window_values)
        sum_sq_diff = np.sum((window_values - mean) ** 2)
        roll_std[i] = np.sqrt(sum_sq_diff / (window - 1))

    return roll_std

USE_DASK = False
import dask.dataframe as dd
def generate_historical_features(df):
    def __generate_historical_features(df):
        print("generate_historical_features")
        target_cols = ['wap', 'match_balance']
        if USE_INDEX:
            target_cols.append('index_mean_wap')
            target_cols.append('index_mean_match_balance')
        if USE_REVEALED_TARGETS:
            target_cols.append('revealed_target')

        grouped = df.groupby(['stock_id', 'date_id'])

        for col in target_cols:
            for window in [3, 5, 7]:
                col_diff_name = f"{col}_diff_{window}"
                df[col_diff_name] = grouped[col].diff(window)
                #df[col_diff_name] = grouped[col].transform(lambda x: cal_diff(x.values, window))

                col_vix_name = f"{col}_vix_{window}"

                if col == 'revealed_target':
                    offset = 10
                else:
                    offset = 0
                
                #df[col_vix_name] = grouped[col].transform(lambda x: np.log(x).diff().rolling(window).std())
                df[col_vix_name] = grouped[col].transform(lambda x: cal_vix(x.values, window))
                #df[col_vix_name] = grouped[col].apply(lambda x: np.log(x + 100).diff().rolling(2).std()).reset_index()[col]

        return df

    # gpu, dskでも速度が出ないので、cpuで実行
    """
    if DEVICE == 'gpu':
        import cudf
        df = cudf.from_pandas(df)
        df = __generate_historical_features(df)
        df = df.to_pandas()
    else:
        if USE_DASK:
            df = dd.from_pandas(df, npartitions=4)  # npartitionsは使用するコアの数に応じて調整
            df = df.set_index('stock_id')
            df = __generate_historical_features(df)
            df = df.compute()
        else:
            df = __generate_historical_features(df)
    """
    df = __generate_historical_features(df)

    df = df.replace([np.inf, -np.inf], 0)
    return df

# サブセットを処理する関数
def subset_generate_historical_features(df_subset):
    return generate_historical_features(df_subset)

# 並列処理を実行する関数
def parallel_generate_historical_features(df, num_threads=NUM_THREADS):
    # DataFrameを 'stock_id' でグループ化
    grouped = df.groupby('stock_id')

    # 並列処理の実行
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = executor.map(subset_generate_historical_features, [group for _, group in grouped])

    # 結果の統合
    results = pd.concat(results)
    return results

def generate_index_features(df):
     # Calculating mean and std for 'wap' and 'match_balance'
    wap_stats = df.groupby(['date_id', 'seconds_in_bucket'])['wap'].agg(['mean', 'std']).reset_index()
    match_balance_stats = df.groupby(['date_id', 'seconds_in_bucket'])['match_balance'].agg(['mean', 'std']).reset_index()

    # Adding prefix and suffix
    wap_stats = wap_stats.add_prefix('index_').add_suffix('_wap')
    match_balance_stats = match_balance_stats.add_prefix('index_').add_suffix('_match_balance')

    # Adjusting column names for merging
    wap_stats = wap_stats.rename(columns={'index_date_id_wap': 'date_id', 'index_seconds_in_bucket_wap': 'seconds_in_bucket'})
    match_balance_stats = match_balance_stats.rename(columns={'index_date_id_match_balance': 'date_id', 'index_seconds_in_bucket_match_balance': 'seconds_in_bucket'})

    # Merging with the original dataframe
    df = df.merge(wap_stats, on=['date_id', 'seconds_in_bucket'], how='left')
    df = df.merge(match_balance_stats, on=['date_id', 'seconds_in_bucket'], how='left')

    return df

def generate_normalized_features(df, is_train):
    print("generate_normalized_features")
    if is_train:
        df['n_target'] = (df['target'] - global_target['mean']) / global_target['std']
    df['n_wap'] = (df['wap'] - global_wap['mean']) / global_wap['std']
    df['n_match_balance'] = (df['match_balance'] - global_mathch_balance['mean']) / global_mathch_balance['std']
    df['n_reference_price'] = (df['reference_price'] - global_reference_price['mean']) / global_reference_price['std']
    
    df = reduce_mem_usage(df, "generate_normalized_features")
    return df

def generate_enhance_features(df, is_train=False):
    print("generate_enhance_features")
    if is_train:
        if USE_REVEALED_TARGETS:
            print("Use revealed targets")
            df[f"revealed_target"] = df.groupby(['stock_id', 'seconds_in_bucket'])['target'].shift(1)
            df = df.dropna(subset=["revealed_target"])
            df = default_sort(df)
        else:
            print("Dosent't use revealed targets")
    if USE_INDEX:
        print("Use index")
        current_time = time.time()
        df = generate_index_features(df)
        print(f"generate_index_features {time.time() - current_time:.2f} [sec]")
    current_time = time.time()
    
    if is_train:
        df = parallel_generate_historical_features(df)
    else:
        df = generate_historical_features(df)
    print(f"generate_historical_features {time.time() - current_time:.2f} [sec]")

    df = df.reset_index(drop=True)
    df = default_sort(df)
    df = reduce_mem_usage(df, "generate_enhance_features")
    collect()
    return df

Step2. Generate enhanced features
CPU times: user 383 ms, sys: 31.7 ms, total: 414 ms
Wall time: 525 ms


In [13]:
df_train = generate_enhance_features(df_train, is_train=True)
generated_feature_name = list(set(df_train.columns) - set(prev_cols))
features += generated_feature_name
print(generated_feature_name)

collect()
print(GetMemUsage())

generate_enhance_features
Use revealed targets
Use index
generate_index_features 3.16 [sec]
generate_historical_featuresgenerate_historical_features

generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_featuresgenerate_historical_features

generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_historical_features
generate_

In [14]:
if IS_DEBUG:
    df_train.to_csv(f"{BASE_OUTPUT_PATH}/df_train.csv", index=False)

# Feature selection

In [15]:
# feature selection
if not USE_ALL_FEATUTES:
    features  = [
        "revealed_target",
        "wap_diff_1",
        "index_mean_wap_diff_1",
        "seconds_in_bucket",
        "stock_id",
    ]
#df_valid = df_train["target"]
#df_train = df_train[features]
#if USE_REVEALED_TARGETS:
#    features.remove("revealed_target")
features

['stock_id',
 'seconds_in_bucket',
 'imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'near_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'all_sizes_skew',
 'spread_intensity',
 'reference_price_near_price_imb',
 'global_ptp_price',
 'global_std_price',
 'all_prices_mean',
 'reference_price_bid_price_imb',
 'matched_size_bid_size_imbalance_size_imb2',
 'far_price_ask_price_imb',
 'price_spread',
 'match_balance',
 'far_price_near_price_imb',
 'global_median_price',
 'reference_price_ask_price_imb',
 'bid_price_wap_imb',
 'global_median_size',
 'near_price_bid_price_imb',
 'ask_price_wap_imb',
 'seconds',
 'all_sizes_kurt',
 'dow',
 'near_price_ask_price_imb',
 'bid_size_ask_size_imbalance_size_imb2',
 'ask_price_bid_price_wap_imb2',
 'minute',
 'depth_pressure',
 'size_imbalance',
 'market_urgency',
 'global_std_size',
 'all_sizes_mean',
 'ask_price_bid_price_reference_price_imb2',
 'near_price_wap_imb',
 'global_

In [16]:
pd_display_max()
nan_count = df_train[features].isna().sum()
#df_train[features].to_csv('train.csv', index=False)
nan_count = nan_count[nan_count > 0].sort_values(ascending=False)
nan_count

revealed_target_vix_7                        4375930
revealed_target_vix_5                        4109444
revealed_target_vix_3                        3694728
reference_price_far_price_imb                2888134
far_price                                    2888134
far_price_wap_imb                            2888134
far_price_bid_price_imb                      2888134
depth_pressure                               2888134
far_price_near_price_imb                     2888134
far_price_ask_price_imb                      2888134
near_price                                   2851092
near_price_wap_imb                           2851092
reference_price_near_price_imb               2851092
near_price_ask_price_imb                     2851092
near_price_bid_price_imb                     2851092
wap_diff_7                                    665356
match_balance_diff_7                          665356
index_mean_wap_diff_7                         665245
index_mean_match_balance_diff_7               

In [17]:
pd_clear_display_max()

# Train function (lightgbm)

In [18]:
# 📦 Import necessary libraries
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import gc
import os
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import numpy as np
from dataclasses import dataclass
import sys
import shutil
import lightgbm as lgb

from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

@dataclass
class Model:
    booster: lgb.Booster
    fold: int
    feature_importance: pd.DataFrame
    score: float
    best_iteration: int
    train_time: float = None
    weight: float = None
    mem_usage: float = None
    train_func: str = None
    is_latest: bool = False

def train_model(train_x, train_y, val_x, val_y, best_params=None):
    trains = lgb.Dataset(train_x, train_y)
    valids = lgb.Dataset(val_x, val_y, reference=trains)

    verbose_eval = -1
    if best_params is None:
        params = lgb_params
    else:
        params = best_params

    print("Use params:")
    print(params)

    booster = lgb.train(
        params,
        trains,
        valid_sets=valids, # 検証データ
        num_boost_round=num_boost_round,
        keep_training_booster=True,
        callbacks=[
                lgb.early_stopping(stopping_rounds=stopping_rounds, verbose=True),
                lgb.log_evaluation(verbose_eval)
        ]
    )

    del trains, valids
    return booster

def cross_train(df, key, n_splits, features, valid_name, best_params=None):
    """ For Cross Train

    Args:
        df (_type_): _description_
        n_splits (_type_): _description_

    Returns:
        _type_: _description_
    """
    print("----------------------------------------")
    print(f"Cross Train key id {key}: start, shape: {df_train.shape}, n_splits: {n_splits}")
    print(f"num_boost_round: {num_boost_round}, stopping_rounds: {stopping_rounds}, folds: {num_folds}")

    models = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    df.reset_index(drop=True, inplace=True)
    
    for fold, (train_indices, valid_indices) in enumerate(kf.split(df)):
        print(f"----- Train {key}: {fold} start -----")
        now_time = time.time()

        print(f"train_indices: {train_indices}, valid_indices: {valid_indices}")
        X_train, X_valid = df[features].iloc[train_indices], df[features].iloc[valid_indices]
        y_train, y_valid = df[valid_name].loc[train_indices], df[valid_name].loc[valid_indices]
        print(f"X_train: {X_train.shape}, X_valid: {X_valid.shape}, y_train: {y_train.shape}, y_valid: {y_valid.shape}")

        booster = train_model(X_train, y_train, X_valid, y_valid, best_params)
        print(booster.best_score)
        y_valid_pred = booster.predict(X_valid)
        
        score = mean_absolute_error(y_valid, y_valid_pred)
        train_time = time.time() - now_time
        mem_usage = sys.getsizeof(booster) / (1024 * 1024) # MB
        model = Model(booster, fold, booster.feature_importance(), score, booster.best_iteration, train_time, weight= 1 / n_splits, mem_usage=mem_usage, train_func="lightgbm", is_latest=True)
        print(f"{key}: {fold} end, score: {score}, time: {model.train_time}, best_iteration: {model.best_iteration}, memory usage: {model.mem_usage}")
        
        models.append(model)
        
        del X_train, X_valid, y_train, y_valid
        gc.collect()
        print(GetMemUsage())
        print(f"----- Train {key}: {fold} end -----")

    print(f"Cross train {key} model len {len(models)}")
    print("----------------------------------------")
    return key, models

# Train function (optuna)

In [19]:
%%time

import optuna.integration.lightgbm as optuna_lgb
import optuna
import lightgbm
optuna.logging.set_verbosity(optuna.logging.ERROR)

class TunerCVCheckpointCallback(object):
    """Optuna の LightGBMTunerCV から学習済みモデルを取り出すためのコールバック"""

    def __init__(self):
        # Models
        self.models = []
        self.counter = 0

    def get_models(self):
        # https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.Booster.html#lightgbm.Booster
        return self.models

    def __call__(self, env: lightgbm.callback.CallbackEnv):
        """_summary_

        Args:
            env (lightgbm.callback.CallbackEnv): _description_
            "model",
            "params",
            "iteration",
            "begin_iteration",
            "end_iteration",
            "evaluation_result_list"
        """
        print("")

        self.counter += 1
        print("-------------------")
        print(f"Counter: {self.counter}")
        print(f"Iteration: {env.iteration}")
        print(f"Begin_iteration: {env.begin_iteration}")
        print(f"End_iteration: {env.end_iteration}")
        print(f"Evaluation_result_list: {env.evaluation_result_list}")
        print(f"Model best_iteration: {env.model.best_iteration}")
        print("Params: ", env.params)
        #self.models.append(env.model)
        del env

        collect();
        print(GetMemUsage())

def optuna_tuning(df, n_splits, features, valid_name, model_save_path):
    df_train = df[features]
    df_valid = df[valid_name]
    
    trains = optuna_lgb.Dataset(df_train, df_valid)
    
    print("------- Optuna Tuning Start -------")
    now_time = time.time()
    print(f"num_boost_round: {num_boost_round}, stopping_rounds: {stopping_rounds}, folds: {num_folds}")

    folds = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    checkpoint_cb = TunerCVCheckpointCallback()
    
    verbose_eval = 0
    # https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.lightgbm.LightGBMTunerCV.html
    tuner = optuna_lgb.LightGBMTunerCV(
        optuna_params,
        trains,
        num_boost_round=num_boost_round,
        folds=folds,
        show_progress_bar=False,
        return_cvbooster=True,
        verbosity=-1,
        model_dir=model_save_path,
        optuna_seed=seed,
        time_budget=OPTUNA_TIME_BUDGET,
        callbacks=[
                lgb.early_stopping(stopping_rounds=stopping_rounds, verbose=True),
                lgb.log_evaluation(verbose_eval),
                checkpoint_cb
        ]
    )
    
    tuner.run()
    best_params = tuner.best_params
    
    print("Params: ")
    for key, value in best_params.items():
        print(" {}: {}".format(key, value))

    print("")
    print("len(tuner.study.trials): ", len(tuner.study.trials))
    #print("len(checkpoint_cb.cv_boosters): ", len(checkpoint_cb.models))
    print("Tuner best_params", tuner.best_params)
    print("Tuner best score: ", tuner.best_score)
   
    # 最も良かったパラメータをキーにして学習済みモデルを取り出す
    best_booster = tuner.get_best_booster()
    score = -1
    train_time = time.time() - now_time
    mem_usage = sys.getsizeof(best_booster) / (1024 * 1024) # MB
    feature_importance = np.mean(best_booster.feature_importance(), axis=0)

    best_model = Model(best_booster, 1, feature_importance, score, best_booster.best_iteration, train_time, weight= 1, mem_usage=mem_usage, train_func="optuna_lgb")
    print("------- Optuna Tuning End -------")
    return best_params, best_model


CPU times: user 4.3 ms, sys: 0 ns, total: 4.3 ms
Wall time: 7.01 ms


# Training

In [20]:
%%time

KEY = "-1"

# Train
best_params = None
key_models = None
if USE_OPTUNA:
    model_save_base_path = f"{BASE_OUTPUT_PATH}/model"
    if os.path.exists(model_save_base_path):
        print(f"{model_save_base_path} already exists, clean up it.")
        shutil.rmtree(model_save_base_path)
    os.makedirs(model_save_base_path)
    print(f"model_save_base_path: {model_save_base_path}")

    best_params, best_model = optuna_tuning(df=df_train, n_splits=num_folds, features=features, valid_name="target", model_save_path=model_save_base_path)
    key_models = [(KEY, [best_model])]
else:
    #key_models = df_train.groupby("seconds_in_bucket").apply(lambda x: cross_train(df=x, key=x.name, n_splits=num_folds, feature_name=feature_name, valid_name="target", best_params=best_params))
    key_models = [cross_train(df_train, key=KEY, n_splits=num_folds, features=features, valid_name="target", best_params=best_params)]
    if IS_USE_SAVED_MODEL:
        model_save_base_path = f"{BASE_OUTPUT_PATH}/model"
        if os.path.exists(model_save_base_path):
            print(f"{model_save_base_path} already exists, clean up it.")
            shutil.rmtree(model_save_base_path)
        os.makedirs(model_save_base_path)

        key_model_paths = []
        for key, models in key_models:
            model_save_path = f"{model_save_base_path}/{key}"
            os.makedirs(model_save_path)
            model_paths = []
            for model in models:
                model_save_fullpath = f"{model_save_path}/model_{key}_{model.fold}.txt"
                model.model.save_model(model_save_fullpath)
                model_paths.append(model_save_fullpath)
            key_model_paths.append((key, model_paths))

        model_dict_saved = {key: model_paths for key, model_paths in key_model_paths}
        print(model_dict_saved)


model_dict = {key: model for key, model in key_models}
collect()
print(GetMemUsage())

----------------------------------------
Cross Train key id -1: start, shape: (5226892, 104), n_splits: 5
num_boost_round: 5000, stopping_rounds: 100, folds: 5
----- Train -1: 0 start -----
train_indices: [      1       2       4 ... 5226889 5226890 5226891], valid_indices: [      0       3       8 ... 5226873 5226877 5226885]
X_train: (4181513, 100), X_valid: (1045379, 100), y_train: (4181513,), y_valid: (1045379,)
Use params:
{'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'device': 'gpu', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 31, 'feature_fraction': 0.8, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'seed': 2023}
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[5000]	valid_0's rmse: 8.61912
defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict([('rmse', 8.61912268081489)])})
-1: 0 end, score: 5.98416886

# Additional training by important features

In [21]:
%%time

additional_features = ['reference_price', 'match_balance_diff_5', 'wap', 'global_std_price',
       'all_sizes_skew', 'matched_size_bid_size_ask_size_imb2', 'ask_price',
       'index_mean_wap_diff_7', 'seconds_in_bucket', 'mid_price',
       'ask_price_bid_price_reference_price_imb2', 'wap_vix_7', 'wap_vix_3',
       'all_sizes_std', 'global_median_price', 'volume', 'all_sizes_mean',
       'revealed_target', 'wap_diff_7', 'global_ptp_size',
       'reference_price_wap_imb', 'bid_price_wap_reference_price_imb2',
       'stock_id', 'global_median_size']
additional_models = []

if USE_ADDITIONAL_TRAIN:
   print("Additional Train")
   key_models = [cross_train(df_train, key=KEY, n_splits=num_folds, features=features, valid_name="target", best_params=best_params)]
   key_additional_models = [cross_train(df_train, key=KEY, n_splits=num_folds, features=additional_features, valid_name="target", best_params=best_params)]
   additional_model_dict = {key: model for key, model in key_additional_models}

collect()
print(GetMemUsage())

Additional Train
----------------------------------------
Cross Train key id -1: start, shape: (5226892, 104), n_splits: 5
num_boost_round: 5000, stopping_rounds: 100, folds: 5
----- Train -1: 0 start -----
train_indices: [      1       2       4 ... 5226889 5226890 5226891], valid_indices: [      0       3       8 ... 5226873 5226877 5226885]
X_train: (4181513, 100), X_valid: (1045379, 100), y_train: (4181513,), y_valid: (1045379,)
Use params:
{'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'device': 'gpu', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 31, 'feature_fraction': 0.8, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'seed': 2023}
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[5000]	valid_0's rmse: 8.61706
defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict([('rmse', 8.617057389795058)])})
-1: 0 end,

# Update model using test

In [22]:
%%time

# global train cache for continuous update
global_train_cache = df_train.copy()
# origin 0 is train, 1 is test, 2 is revaled
global_train_cache['origin'] = 0
date_duration = DATA_COUNT_IN_SAME_BUCKET * continuos_dataset_span
print("date_duration", date_duration)

def update_global_train_cache(df, origin, valid_key: str = 'target'):
    global global_train_cache
    df['origin'] = origin
    print("update_global_train_cache")
    global_train_cache = pd.concat([global_train_cache, df], axis=0)
    global_train_cache = global_train_cache.dropna(subset=['target'])
    global_train_cache = global_train_cache.sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id', 'origin'])
    global_train_cache = global_train_cache.drop_duplicates(['date_id', 'seconds_in_bucket', 'stock_id'], keep='last')
    global_train_cache = global_train_cache.reset_index(drop=True)
    global_train_cache = global_train_cache.groupby(['stock_id']).tail(date_duration).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)

    global_train_cache = reduce_mem_usage(global_train_cache, 'global_train_cache')
    print(f"Updated global_train_cache, len: ", len(global_train_cache))
    if IS_DEBUG:
        print(len(global_train_cache))
        if USE_REVEALED_TARGETS:
            cdf = global_train_cache[['date_id', 'seconds_in_bucket', 'stock_id', 'origin', 'revealed_target', 'target']]
        else:
            cdf = global_train_cache[['date_id', 'seconds_in_bucket', 'stock_id', 'origin', 'target']]
        print(cdf)

def update_models(df, models, features, valid_name):
    """ For Update Model

    Args:
        df (_type_): _description_
        n_splits (_type_): _description_

    Returns:
        _type_: _description_
    """

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    df.reset_index(drop=True, inplace=True)
    
    for fold, (train_indices, valid_indices) in enumerate(kf.split(df)):
        print(f"{key}: {fold} update")
        now_time = time.time()
        X_train, X_valid = df[features].iloc[train_indices], df[features].iloc[valid_indices]
        y_train, y_valid = df[valid_name].loc[train_indices], df[valid_name].loc[valid_indices]
        trains = lgb.Dataset(X_train, y_train, free_raw_data=False)
        valids = lgb.Dataset(X_valid, y_valid, reference=trains)

        print(f"X_train: {X_train.shape}, X_valid: {X_valid.shape}, y_train: {y_train.shape}, y_valid: {y_valid.shape}")
        models[fold].booster.update(trains)

def keep_train_models(df, models, features, valid_name, is_append=False):
    print(f"----------------- keep_train_models, is_append: {is_append}, model len: {len(models)}, df len: {len(df)} ---------------------")
    train_x = df[features]
    train_y = df[valid_name]
    trains = lgb.Dataset(train_x, train_y, free_raw_data=False)
    verbose_eval = -1

    counter = 0
    r_models = []
    if IS_DEBUG:
        print(f"Re-train dataset:")
        if USE_REVEALED_TARGETS:
            print(df[['date_id', 'seconds_in_bucket', 'stock_id', 'target', 'revealed_target']])
        else:
            print(df[['date_id', 'seconds_in_bucket', 'stock_id', 'target']])
    for model in models:
        print(f"---- train start, counter: {counter} ----")
        if model.is_latest:
            print("Update latest model")
            now_time = time.time()
            booster = lgb.train(
                lgb_params,
                trains,
                num_boost_round=update_num_boost_round,
                keep_training_booster=True,
                init_model=model.booster,
            )
            train_time = time.time() - now_time
            updated_model = Model(
                booster=booster,
                fold=1,
                best_iteration=booster.best_iteration, 
                feature_importance=booster.feature_importance(),
                score=-1, 
                train_time=train_time, 
                weight=-1, 
                mem_usage=-1,
                is_latest=True,
                train_func="lightgbm update by test")
            r_models.append(updated_model)
            if is_append:
                print("Adding previous model")
                model.is_latest = False
                r_models.append(model)
        else:
            print("Dose not latest, just append")
            r_models.append(model)
        counter = counter + 1
    print(f"---- train end, train time: {train_time}, updated model len {len(r_models)} ----")
    return r_models

if USE_CONTINUOUS_UPDATE:
    try:
        print("Update model with test date")
        df_test = load_test_dataset()
        if IS_MIN_LEARN:
            print("MIN LEARN MODE :", TARGET_STOCK_IDS)
            df_test = df_test[df_test["stock_id"].isin(TARGET_STOCK_IDS)]
        df_test = generate_basic_features(df_test)
        df_test = generate_enhance_features(df_test)
        update_global_train_cache(df_test, 1)
        #model_dict[KEY] = keep_train_models(global_train_cache, model_dict[KEY], features, "target", is_append=True)
        print("Update model with test date end")
    except Exception as e:
        print("Cannot get test date", e)

collect()
print(GetMemUsage())


date_duration 1100
Update model with test date
generate_enhance_features
Use index
generate_index_features 0.02 [sec]
generate_historical_features
generate_historical_features 0.62 [sec]
update_global_train_cache
Updated global_train_cache, len:  220000
220000
        date_id  seconds_in_bucket  stock_id  origin  revealed_target  \
0           461                  0         0       0        -2.830029   
1           461                  0         1       0        11.060238   
2           461                  0         2       0         2.080202   
3           461                  0         3       0         2.720356   
4           461                  0         4       0        -5.499721   
...         ...                ...       ...     ...              ...   
219995      480                540       195       0         1.599789   
219996      480                540       196       0        -8.440018   
219997      480                540       197       0         5.149841   
219998   

In [23]:
# Show results

for key, models in model_dict.items():
    print(f"Key: {key}, model len: {len(models)}")
    data = []
    for model in models:
        score = model.score
        best_iteration = model.best_iteration
        fold = model.fold
        train_time = model.train_time
        data.append({"key": key, "fold": fold, "score": score, "best_iteration": best_iteration, "train_time": train_time})

    df_model = pd.DataFrame(data)
    print(df_model.describe())

Key: -1, model len: 5
           fold     score  best_iteration  train_time
count  5.000000  5.000000             5.0    5.000000
mean   2.000000  5.988596          5000.0  458.897379
std    1.581139  0.009782             0.0    6.566664
min    0.000000  5.976397          5000.0  452.205882
25%    1.000000  5.984169          5000.0  455.560069
50%    2.000000  5.986117          5000.0  457.106990
75%    3.000000  5.994567          5000.0  460.142302
max    4.000000  6.001732          5000.0  469.471649


In [24]:
# Check model quality
data = []

for key, i_models in model_dict.items():
    for model in i_models:
        score = model.score
        best_iteration = model.best_iteration
        fold = model.fold
        train_time = model.train_time
        data.append({"key": key, "fold": fold, "score": score, "best_iteration": best_iteration, "train_time": train_time})

df_model = pd.DataFrame(data)
df_model.describe()

,fold,score,best_iteration,train_time
count,5.000000,5.000000,5.0,5.000000
mean,2.000000,5.988596,5000.0,458.897379
std,1.581139,0.009782,0.0,6.566664
min,0.000000,5.976397,5000.0,452.205882
25%,1.000000,5.984169,5000.0,455.560069
50%,2.000000,5.986117,5000.0,457.106990
75%,3.000000,5.994567,5000.0,460.142302
max,4.000000,6.001732,5000.0,469.471649


In [25]:
# Initialize an empty DataFrame for aggregated importances
aggregated_importance = pd.DataFrame(index=features, columns=['importance'])

# Aggregate the importances from each model
for key, i_models in model_dict.items():
    for model in i_models:
        importance = pd.DataFrame({'feature': features, 'importance': model.feature_importance})
        aggregated_importance = aggregated_importance.add(importance.set_index('feature'), fill_value=0)

aggregated_importance['importance'] /= len(df_model)

pd_display_max()
# Sort the features by importance
aggregated_importance = aggregated_importance.sort_values(by='importance', ascending=False)
aggregated_importance

,importance
index_mean_match_balance,4405.4
global_std_price,4187.8
index_mean_wap,4014.6
index_std_wap,3808.4
index_std_match_balance,3754.6
all_sizes_mean,2915.4
matched_size,2745.2
global_ptp_price,2705.8
match_balance,2636.4
global_median_price,2606.8


# Dataset types

In [26]:
features_types = df_train[features].dtypes
features_types

stock_id                                       int16
seconds_in_bucket                              int16
imbalance_size                               float32
imbalance_buy_sell_flag                         int8
reference_price                              float32
matched_size                                 float32
far_price                                    float32
near_price                                   float32
bid_price                                    float32
bid_size                                     float32
ask_price                                    float32
ask_size                                     float32
wap                                          float32
all_sizes_skew                               float32
spread_intensity                             float32
reference_price_near_price_imb               float32
global_ptp_price                             float32
global_std_price                             float32
all_prices_mean                              f

In [27]:
def convert_dtypes(df):
    df_types = df[features].dtypes
    different_types = [col for col in df_types.index if col in features_types and df_types[col] != features_types[col]]
    print(f"Different Types: {different_types}")
    return different_types

def update_dtypes_by_origin(df):
    diff_types = convert_dtypes(df)
    for col in diff_types:
        df[col] = df[col].astype(features_types[col])
    return df

# Clear trains

In [28]:
# Clean up
pd_clear_display_max()
del key_models
if IS_USE_SAVED_MODEL:
    print("Delete model_dict")
    del model_dict
del df_train
collect()
print(GetMemUsage())

RAM memory GB usage = 18.81


# Infer

In [29]:
%%time

y_min, y_max = -64, 64

# 📉 Define a function to adjust prices based on volumes
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)  # 🧮 Calculate standard error based on volumes
    step = np.sum(prices) / np.sum(std_error)  # 🧮 Calculate the step size based on prices and standard error
    out = prices - std_error * step  # 💰 Adjust prices by subtracting the standardized step size
    return out

def zero_clip(df, predictions):
    # Adjust the predictions based on the order book imbalance
    zerosum_predictions = zero_sum(predictions, df['bid_size'] + df['ask_size'])
    clipped_predictions = np.clip(zerosum_predictions, y_min, y_max)
    clipped_predictions.replace([np.nan, np.inf, -np.inf], 0, inplace=True)
    clipped_predictions = clipped_predictions.astype('float64').values  
    return clipped_predictions

def predict_scaler(df):
    pred_mean = df['pred'].mean()
    pred_std = df['pred'].std()
    
    target_mean = global_target['mean']
    target_std = global_target['std']

    df['scaled_pred'] = (df['pred'] - pred_mean) / pred_std * target_std + target_mean
    return df

def model_infer(key, df_feat, use_additional_model=False):
    def predictor(boosters):
        print(f"Predictor target models len {len(boosters)}")
        #print(f"Predictor Feat len {len(df_feat)}")
        if USE_OPTUNA:
            predictions_list = [np.mean(booster.predict(df_feat), 0) for booster in boosters]
        else:
            predictions_list = [booster.predict(df_feat) for booster in boosters]
        predictions = np.mean(predictions_list, 0)
        std_predictions = np.std(predictions_list, 0)
        #print("std_predictions", std_predictions)
        return predictions
    
    if IS_USE_SAVED_MODEL:
        model_paths = model_dict_saved[key]
        models = [lgb.Booster(model_file=model_path) for model_path in model_paths]
        predictions = predictor(models)
        del models
    else:
        if USE_ADDITIONAL_TRAIN and use_additional_model:
            print("Use additional model")
            boosters = [m.booster for m in additional_model_dict[key]]
        else:
            boosters = [m.booster for m in model_dict[key]]
        predictions = predictor(boosters)
    collect()
    return predictions

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 12.6 µs


In [30]:
%%time

predictions = []
df_cache = pd.DataFrame()
df_cache_with_features = pd.DataFrame()
df_result = pd.DataFrame()

df_revealed_targets = pd.DataFrame()

if IS_INFER:
    if IS_LOCAL or IS_DEBUG:
        print("Infer Local")
        env = make_env()
    else:
        print("Infer Submission")
        import optiver2023
        env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 1

    try:
        for (test, revealed_targets, sample_prediction) in iter_test:
            now_time = time.time()
            print(f"------- counter {counter} start -------")

            # Add revealed target as target for counituous update
            copy_revealed_targets = revealed_targets.copy()
            copy_revealed_targets = copy_revealed_targets.dropna()
            print("copy_revealed_targets len", len(copy_revealed_targets))

            if len(copy_revealed_targets) > 0:
                print("Update revealed_targets")
                copy_revealed_targets['revealed_date_id'] = copy_revealed_targets['revealed_date_id'].astype(int).astype(str)
                copy_revealed_targets['date_id'] = copy_revealed_targets['date_id'].astype(int).astype(str)
                copy_revealed_targets['seconds_in_bucket'] = copy_revealed_targets['seconds_in_bucket'].astype(int).astype(str)
                copy_revealed_targets['stock_id'] = copy_revealed_targets['stock_id'].astype(int).astype(str)  # Converting to int first to remove any decimal points
                copy_revealed_targets['revealed_row_id'] = copy_revealed_targets['revealed_date_id'] + '_' + copy_revealed_targets['seconds_in_bucket'] + '_' + copy_revealed_targets['stock_id']
                copy_revealed_targets['row_id'] = copy_revealed_targets['date_id'] + '_' + copy_revealed_targets['seconds_in_bucket'] + '_' + copy_revealed_targets['stock_id']
                copy_revealed_targets['revealed_target'] = copy_revealed_targets['revealed_target'].astype('float32')

                df_revealed_targets = pd.concat([df_revealed_targets, copy_revealed_targets], ignore_index=True, axis=0)
                df_revealed_targets = df_revealed_targets.groupby(['stock_id']).tail(DATA_COUNT_IN_SAME_BUCKET * 2)
                df_revealed_targets = reduce_mem_usage(df_revealed_targets, 'df_revealed_targets')
                
            df_cache = pd.concat([df_cache, test], ignore_index=True, axis=0)

            if IS_MIN_LEARN:
                print("MIN LEARN MODE :", TARGET_STOCK_IDS)
                df_cache = df_cache[df_cache["stock_id"].isin(TARGET_STOCK_IDS)]

            if counter > 0:
                # Clear cache data, tailはhistoricalで作成な分のみ残す
                df_cache = df_cache.groupby(['stock_id']).tail(10)
                df_cache = default_sort(df_cache)
                print(f"df_cache len {len(df_cache)}")

            # USE_REVEALED_TARGETSが有効の時、cacheのrow_idとdf_revealed_targetsのrow_idをleft joinする
            if USE_REVEALED_TARGETS:
                df_r = df_revealed_targets[['row_id', 'revealed_target']]
                df_cache = pd.merge(df_cache, df_r, how='left', on='row_id')
                df_cache[['date_id', 'seconds_in_bucket', 'stock_id', 'revealed_target']]

            # Generate features
            df_valid = df_cache.copy()
            df_valid = generate_basic_features(df_valid)
            df_valid = generate_enhance_features(df_valid)
            df_valid = reduce_mem_usage(df_valid, 'df_valid')

            # testの分のみの長さを抽出
            if IS_MIN_LEARN:
                df_valid = df_valid[-len(TARGET_STOCK_IDS):].reset_index(drop=True)
            else:
                df_valid = df_valid[-len(test):].reset_index(drop=True)

            df_cache_with_features = pd.concat([df_cache_with_features, df_valid], ignore_index=True, axis=0)

            # It faults due to test is iterator
            #seconds_in_bucket = test['seconds_in_bucket'][0]
            #print(f"prdict: {test['date_id'][0]}, {seconds_in_bucket}")

            seconds_in_bucket = df_valid['seconds_in_bucket'][0] / 10
            date_id = df_valid['date_id'][0]
            print(f"date_id: {date_id},  seconds_in_bucket: {seconds_in_bucket}")

            if counter > 0:
                # Clear cache data, tailはhistoricalで作成な分のみ残す
                df_cache_with_features = df_cache_with_features.groupby(['stock_id']).tail(DATA_COUNT_IN_SAME_BUCKET * 2)
                df_cache_with_features = default_sort(df_cache_with_features)
                print(f"df_cache_with_features len {len(df_cache_with_features)}")

            # Update global train cache
            if USE_CONTINUOUS_UPDATE  and ((seconds_in_bucket + 1) % DATA_COUNT_IN_SAME_BUCKET == 0):
                print("Update global train cache")
                df_r = df_revealed_targets[['revealed_row_id', 'revealed_target']]
                df_r.rename(columns={'revealed_target': 'target'}, inplace=True)
                df_r.rename(columns={'revealed_row_id': 'row_id'}, inplace=True)
                df_update = pd.merge(df_cache_with_features, df_r, how='left', on='row_id')
                update_global_train_cache(df_update, 2)

            # Update model
            if ((seconds_in_bucket + 1)  % (DATA_COUNT_IN_SAME_BUCKET * continuos_train_span) == 0) and USE_CONTINUOUS_UPDATE:
                print("Update model with revealed_target date start")
                train_now_time = time.time()
                model_dict[KEY] = keep_train_models(global_train_cache, model_dict[KEY], features, "target", True)
                print(f"ReTrain Time: {time.time() - train_now_time}")

            # Predict
            predictions = model_infer(KEY, df_valid[features])
            scaled_predictions = zero_clip(df_valid, predictions)

            df_valid['pred'] = predictions
            df_valid['scaled_pred'] = scaled_predictions
            print("prediction average", np.mean(predictions))

            if USE_REVEALED_TARGETS:
                additional_predictions = model_infer(KEY, df_valid[additional_features], use_additional_model=True)
                additional_scaled_predictions = zero_clip(df_valid, additional_predictions)
                df_valid['additional_pred'] = additional_predictions
                df_valid['additional_scaled_pred'] = additional_scaled_predictions
                print("additional prediction average", np.mean(additional_scaled_predictions))
            # For save
            if IS_DEBUG:
                df_result = pd.concat([df_result, df_valid], ignore_index=True, axis=0)

            # Submit
            if not IS_MIN_LEARN:
                print("Submit prediction")
                sample_prediction['target'] = scaled_predictions
                env.predict(sample_prediction)
            else:
                print("Submit dummy prediction")
                sample_prediction['target'] = 0
                env.predict(sample_prediction)

            # Clean up
            execution_time = time.time() - now_time
            df_cache = df_cache.drop('revealed_target', axis=1)
            del df_valid
            collect()
            print(GetMemUsage())
            print(f"------- counter {counter}, execution_time {execution_time} end -------")
            counter += 1
    except Exception as e:
        print("Error", e)

Infer Local
------- counter 1 start -------
copy_revealed_targets len 11000
Update revealed_targets
df_cache len 200
generate_enhance_features
Use index
generate_index_features 0.01 [sec]
generate_historical_features
generate_historical_features 0.25 [sec]
date_id: 478,  seconds_in_bucket: 0.0
df_cache_with_features len 200
Predictor target models len 5
prediction average 0.7796153813883541
Use additional model
Predictor target models len 5
additional prediction average 1.357372657384559e-08
Submit prediction
RAM memory GB usage = 18.81
------- counter 1, execution_time 1.9559648036956787 end -------
------- counter 2 start -------
copy_revealed_targets len 0
df_cache len 400
generate_enhance_features
Use index
generate_index_features 0.01 [sec]
generate_historical_features
generate_historical_features 0.25 [sec]
date_id: 478,  seconds_in_bucket: 1.0
df_cache_with_features len 400
Predictor target models len 5
prediction average 0.13066907231148606
Use additional model
Predictor target

In [31]:
if IS_DEBUG:
    df_cache_with_features.to_csv(f"{BASE_OUTPUT_PATH}/df_cache_with_features.csv", index=False)
    df_result.to_csv(f"{BASE_OUTPUT_PATH}/result.csv", index=False)

In [32]:
if IS_DEBUG:
    df_revealed_targets = pd.read_csv(REVEALED_TARGETS_FILE)
    df_revealed_targets = df_revealed_targets.dropna(subset=['revealed_date_id', 'seconds_in_bucket', 'stock_id'])
    df_revealed_targets['revealed_date_id'] = df_revealed_targets['revealed_date_id'].astype(int).astype(str)
    df_revealed_targets['seconds_in_bucket'] = df_revealed_targets['seconds_in_bucket'].astype(int).astype(str)
    df_revealed_targets['stock_id'] = df_revealed_targets['stock_id'].astype(int).astype(str)  # Converting to int first to remove any decimal points

    # Concatenate the columns
    df_revealed_targets['row_id'] = df_revealed_targets['revealed_date_id'] + '_' + df_revealed_targets['seconds_in_bucket'] + '_' + df_revealed_targets['stock_id']
    
    df_pred = df_result[['row_id', 'scaled_pred']]
    df = pd.merge(df_pred, df_revealed_targets, how='left', on='row_id')

    df = df.rename(columns={'revealed_target': 'target'})
    df = df.dropna(subset=['target'])
    df['score'] = (df['scaled_pred'] - df['target']).abs()

    #df['score'] = mean_absolute_error(df['scaled_pred'], df['target'])
    df = df[['row_id', 'scaled_pred', 'target', 'score']]
    print("score")
    print(df.describe())

    if USE_ADDITIONAL_TRAIN:
        df_pred = df_result[['row_id', 'additional_scaled_pred']]
        df = pd.merge(df_pred, df_revealed_targets, how='left', on='row_id')

        df = df.rename(columns={'revealed_target': 'target'})
        df = df.dropna(subset=['target'])
        df['score'] = (df['additional_scaled_pred'] - df['target']).abs()

        #df['score'] = mean_absolute_error(df['scaled_pred'], df['target'])
        print("additional_pred")
        df = df[['row_id', 'additional_scaled_pred', 'target', 'score']]
        print(df.describe())

score
        scaled_pred        target         score
count  22000.000000  22000.000000  22000.000000
mean       0.005682     -0.171779      5.070527
std        2.907103      8.208760      5.046819
min      -64.000000   -144.379730      0.000889
25%       -0.754184     -3.932267      1.713513
50%        0.079279     -0.060201      3.673265
75%        0.851452      3.730059      6.840511
max       55.841760     77.129600     80.379730
additional_pred
       additional_scaled_pred        target         score
count            22000.000000  22000.000000  22000.000000
mean                 0.005521     -0.171779      5.164920
std                  2.623771      8.208760      5.185243
min                -64.000000   -144.379730      0.000479
25%                 -0.621228     -3.932267      1.740845
50%                  0.069669     -0.060201      3.735483
75%                  0.724189      3.730059      6.923081
max                 46.050495     77.129600     80.379730
